In [5]:
import pandas as pd
from project_config import PREPROCESSED_2012_DATA_FILE, PREPROCESSED_2022_DATA_FILE
from utils.file_encoding_detector import detect_encoding
from utils.read_csv_file import read_csv_file

file_2012 = PREPROCESSED_2012_DATA_FILE
file_2022 = PREPROCESSED_2022_DATA_FILE

df_1 = read_csv_file(file_2012)
df_2 = read_csv_file(file_2022)

df_2.group.nunique()

673

In [ ]:
from project_config import YEARBOOK_2012_DATA_DIR


source_data_dir = YEARBOOK_2012_DATA_DIR

files_to_check_against = list(source_data_dir.glob("*.xls*"))

print(files_to_check_against)

In [2]:
from project_config import TRAINING_DATA_FILE, TRAINING_INFERENCE_DATA_FILE
from utils.read_csv_file import read_csv_file

df1 = read_csv_file(TRAINING_DATA_FILE)
df2 = read_csv_file(TRAINING_INFERENCE_DATA_FILE)

print(len(df1), len(df2))

17403 98742


In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from gensim.models import KeyedVectors

# Load pre-trained Word2Vec model (Google News dataset)
word_vectors = KeyedVectors.load_word2vec_format(
    "GoogleNews-vectors-negative300.bin", binary=True
)

# Words to visualize
words = ["king", "queen", "man", "woman"]
vectors = [word_vectors[word] for word in words]

# Reduce to 2D for visualization
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(vectors)

# Plot
plt.figure(figsize=(8, 8))
for i, word in enumerate(words):
    plt.scatter(reduced_vectors[i, 0], reduced_vectors[i, 1], label=word)
    plt.text(reduced_vectors[i, 0] + 0.02, reduced_vectors[i, 1], word, fontsize=12)
plt.title("Word Embedding Visualization (King, Queen, Man, Woman)")
plt.xlabel("PCA Dimension 1")
plt.ylabel("PCA Dimension 2")
plt.legend()
plt.show()

In [ ]:
import pandas as pd
from project_config import TRAINING_DATA_FILE
from utils.read_csv_file import read_csv_file

df = read_csv_file(TRAINING_DATA_FILE)

,text,yearbook_source,section,group,row_id,is_title,is_empty,original_index,label
0,1-1 Divisions of Administrative Areas in Chin...,2012,General Survey,B0101e,1,yes,no,0,title
1,"EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPT...",2012,General Survey,B0101e,2,no,yes,1,empty
2,"(unit), EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMP...",2012,General Survey,B0101e,3,no,no,2,metadata
3,"EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPT...",2012,General Survey,B0101e,4,no,yes,3,empty
4,"Provinces Autonomous , Number of, Cities, Num...",2012,General Survey,B0101e,5,no,no,4,header
...,...,...,...,...,...,...,...,...,...
17398,"United Kingdom, 0.66, 0.65, 0.78, 0.75, 0.78,...",2022,Appendix II. A Comparison of Indicators of Eco...,E29-15,47,no,no,17398,table_data
17399,"Australia, 1.73, 1.09, 1.31, 1.34, 1.44, 1.45...",2022,Appendix II. A Comparison of Indicators of Eco...,E29-15,48,no,no,17399,table_data
17400,"New Zealand, 2.2, 1.39, 1.41, 1.45, 1.52, 1.5...",2022,Appendix II. A Comparison of Indicators of Eco...,E29-15,49,no,no,17400,table_data
17401,"EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPT...",2022,Appendix II. A Comparison of Indicators of Eco...,E29-15,50,no,yes,17401,empty


# Inspect Final Output Files

In [15]:
import pandas as pd
from utils.read_csv_file import read_csv_file

from project_config import (
    INFERENCE_INPUT_DATA_FILE,
    RAW_INFERENCE_OUTPUT_DATA_FILE,
    CLEANED_INFERENCE_OUTPUT_DATA_FILE,
    TRAINING_DATA_FILE,
    CLEANED_TRAINING_OUTPUT_DATA_FILE,
    COMBINED_CLEANED_OUTPUT_DATA_FILE,
)

df = pd.read_csv(CLEANED_INFERENCE_OUTPUT_DATA_FILE)
blank_cells = df["text"].isnull().sum()
print(f"inference file blank text cells: {blank_cells}")

df = read_csv_file(TRAINING_DATA_FILE)
blank_cells = df["text"].isnull().sum()
print(f"original_training file blank text cells: {blank_cells}")

df = pd.read_csv(CLEANED_TRAINING_OUTPUT_DATA_FILE)
blank_cells = df["text"].isnull().sum()
print(f"cleaned_training file blank text cells: {blank_cells}")

df = pd.read_csv(COMBINED_CLEANED_OUTPUT_DATA_FILE)
blank_cells = df["text"].isnull().sum()
print(f"combined file blank text cells: {blank_cells}")

inference file blank text cells: 0


root - INFO - Detected encoding: ISO-8859-1 with confidence 0.73
root - INFO - File read successfully from C:\github\table_lense\pipeline_data\training_inference\training\training_data_v1.csv


original_training file blank text cells: 0
cleaned_training file blank text cells: 93
combined file blank text cells: 93


In [14]:
import logging
from pathlib import Path
from data_processing.post_process_inference_data import (
    process_inference_training_results,
)
from project_config import TRAINING_DATA_FILE, CLEANED_TRAINING_OUTPUT_DATA_FILE

# Optional: Setup logging if you want to see output
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)


def clean_training_data():
    """
    Re-clean the training input data using the same cleaning pipeline
    used in the inference pipeline.
    """
    input_file = Path(TRAINING_DATA_FILE)
    output_file = Path(CLEANED_TRAINING_OUTPUT_DATA_FILE)

    logging.info(f"Cleaning training data from: {input_file}")
    process_inference_training_results(
        input_file_path=input_file, output_file_path=output_file
    )
    logging.info(f"Cleaned training data saved to: {output_file}")


if __name__ == "__main__":
    clean_training_data()

root - INFO - Cleaning training data from: C:\github\table_lense\pipeline_data\training_inference\training\training_data_v1.csv
data_processing.post_process_inference_data - INFO - Start processing file (C:\github\table_lense\pipeline_data\training_inference\training\training_data_v1.csv)...


[DEBUG] Retrieved username: xzhan
[DEBUG] Log file path: C:\github\table_lense\logs\xzhan_20250322_172204.log
[DEBUG] Logging successfully configured with a single file handler


data_processing.post_process_inference_data - INFO - Detected encoding: ISO-8859-1 with confidence 0.73
data_processing.post_process_inference_data - INFO - File read successfully from C:\github\table_lense\pipeline_data\training_inference\training\training_data_v1.csv
data_processing.post_process_inference_data - INFO - DataFrame loaded:
                                                text  yearbook_source  \
0  1-1  Divisions of Administrative Areas in Chin...             2012   
1  EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPT...             2012   
2  (unit), EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMP...             2012   
3  EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPT...             2012   
4  Provinces  Autonomous , Number of, Cities, Num...             2012   

          section   group  row_id is_title is_empty  original_index     label  
0  General Survey  B0101e       1      yes       no               0     title  
1  General Survey  B0101e       2       no      yes         

## Check training file, inference in, inference embeddings, training + inference

In [16]:
import pickle
from pathlib import Path
from project_config import (
    INFERENCE_EMBEDDINGS_PKL_FILE,
    TRAINING_INFERENCE_DATA_FILE,
    TRAINING_DATA_FILE,
    RAW_INFERENCE_OUTPUT_DATA_FILE,
)
import pandas as pd
from utils.read_csv_file import read_csv_file

partial_cache_path = Path(INFERENCE_EMBEDDINGS_PKL_FILE)

with partial_cache_path.open("rb") as f:
    cache_data = pickle.load(f)

results = cache_data.get("results", [])
total_rows = sum(batch[0].shape[0] for batch in results if hasattr(batch[0], "shape"))
print("Total individual rows in training_embedding pickle file:", total_rows)

training_inference_data = read_csv_file(TRAINING_INFERENCE_DATA_FILE)
training_data = read_csv_file(TRAINING_DATA_FILE)
inference_output_data = read_csv_file(RAW_INFERENCE_OUTPUT_DATA_FILE)

print(f"inference output data: {len(inference_output_data)}")
print(f"training and inference input data: {len(training_inference_data)}")
print(f"training input data: {len(training_data)}")

print()
total = len(training_data) + len(inference_output_data)
print(f"total: {total}")

Total individual rows in training_embedding pickle file: 69634
inference output data: 69634
training and inference input data: 98742
training input data: 17403

total: 87037


## Check for duplicates in different files

In [18]:
import pickle
from pathlib import Path
from project_config import (
    INFERENCE_EMBEDDINGS_PKL_FILE,
    TRAINING_INFERENCE_DATA_FILE,
    TRAINING_DATA_FILE,
    RAW_INFERENCE_OUTPUT_DATA_FILE,
)
import pandas as pd
from utils.read_csv_file import read_csv_file

train_inf_df = read_csv_file(TRAINING_INFERENCE_DATA_FILE)

train_inf_df.original_index.nunique() == len(train_inf_df)

True

### Check inference files

In [6]:
import pandas as pd
from pathlib import Path
import pickle
from project_config import (
    CLEANED_INFERENCE_OUTPUT_DATA_FILE,
    INFERENCE_INPUT_DATA_FILE,
    RAW_INFERENCE_OUTPUT_DATA_FILE,
)
from utils.read_csv_file import read_csv_file


def load_and_validate_data(file_paths):
    """
    Load data from CSV files and validate the 'original_indices' column.

    Args:
        file_paths (dict): A dictionary containing file paths with descriptive keys.

    Returns:
        dict: A dictionary containing loaded DataFrames.
    """
    data = {}
    for key, file_path in file_paths.items():
        df = read_csv_file(file_path)
        data[key] = df

        # Check if 'original_index' column exists
        if "original_index" in df.columns:
            # Check if 'original_index' column is unique
            if df["original_index"].is_unique:
                print(f"'original_index' column in {key} is unique.")
            else:
                print(f"Warning: 'original_index' column in {key} is not unique.")
        else:
            print(f"Warning: 'original_index' column not found in {key}.")
    return data


def print_data_statistics(data):
    """
    Print statistics about the loaded data.

    Args:
        data (dict): A dictionary containing loaded DataFrames.
    """
    for key, df in data.items():
        print(f"{key}: {len(df)}")


# Define file paths
file_paths = {
    "Input Inference": INFERENCE_INPUT_DATA_FILE,
    "Raw Output Inference": RAW_INFERENCE_OUTPUT_DATA_FILE,
    "Cleaned Output Inference": CLEANED_INFERENCE_OUTPUT_DATA_FILE,
}

# Load and validate data
data = load_and_validate_data(file_paths)

# Print data statistics
print_data_statistics(data)

'original_index' column in Input Inference is unique.
'original_index' column in Raw Output Inference is unique.
'original_index' column in Cleaned Output Inference is unique.
Input Inference: 69634
Raw Output Inference: 69634
Cleaned Output Inference: 69634


In [2]:
import pandas as pd
from pathlib import Path
import pickle
from project_config import (
    CLEANED_INFERENCE_OUTPUT_DATA_FILE,
    INFERENCE_INPUT_DATA_FILE,
    RAW_INFERENCE_OUTPUT_DATA_FILE,
    CLEANED_TRAINING_OUTPUT_DATA_FILE,
)
from utils.read_csv_file import read_csv_file


def load_and_validate_data(file_paths):
    """
    Load data from CSV files and validate the 'original_indices' column.

    Args:
        file_paths (dict): A dictionary containing file paths with descriptive keys.

    Returns:
        dict: A dictionary containing loaded DataFrames.
    """
    data = {}
    for key, file_path in file_paths.items():
        df = read_csv_file(file_path)
        data[key] = df

        # Check if 'original_index' column exists
        if "original_index" in df.columns:
            # Check if 'original_index' column is unique
            if df["original_index"].is_unique:
                print(f"'original_index' column in {key} is unique.")
            else:
                print(f"Warning: 'original_index' column in {key} is not unique.")
        else:
            print(f"Warning: 'original_index' column not found in {key}.")
    return data


def print_data_statistics(data):
    """
    Print statistics about the loaded data.

    Args:
        data (dict): A dictionary containing loaded DataFrames.
    """
    for key, df in data.items():
        print(f"{key}: {len(df)}")


def compare_original_index(data, file1, file2):
    """
    Compare the 'original_index' columns between two files.

    Args:
        data (dict): A dictionary containing loaded DataFrames.
        file1 (str): The key of the first file in the data dictionary.
        file2 (str): The key of the second file in the data dictionary.
    """
    # Get the 'original_index' columns from the two files
    original_index1 = set(data[file1]["original_index"])
    original_index2 = set(data[file2]["original_index"])

    # Find the intersection of the two sets
    overlap = original_index1.intersection(original_index2)

    # Print the result
    if overlap:
        print(f"Overlap between '{file1}' and '{file2}': {len(overlap)}")
    else:
        print(f"No overlap between '{file1}' and '{file2}'")


# Define file paths
file_paths = {
    # "Input Inference": INFERENCE_INPUT_DATA_FILE,
    # "Raw Output Inference": RAW_INFERENCE_OUTPUT_DATA_FILE,
    "Cleaned Output Inference": CLEANED_INFERENCE_OUTPUT_DATA_FILE,
    "Cleaned Training": CLEANED_TRAINING_OUTPUT_DATA_FILE,
}

# Load and validate data
data = load_and_validate_data(file_paths)

# Print data statistics
print_data_statistics(data)

# Compare the 'original_index' columns between two files
compare_original_index(data, "Cleaned Output Inference", "Cleaned Training")

'original_index' column in Cleaned Output Inference is unique.
'original_index' column in Cleaned Training is unique.
Cleaned Output Inference: 69634
Cleaned Training: 17403
Overlap between 'Cleaned Output Inference' and 'Cleaned Training': 15497


In [18]:
from project_config import TRAINING_DATA_FILE, INFERENCE_INPUT_DATA_FILE


def has_unique_values(series):
    return series.is_unique


# Define file paths
file_paths = {
    "Input Inference": INFERENCE_INPUT_DATA_FILE,
    "Training": TRAINING_DATA_FILE,
}
# Load and validate data
data = load_and_validate_data(file_paths)
compare_original_index(data, "Input Inference", "Training")

'original_index' column in Input Inference is unique.
'original_index' column in Training is unique.
Overlap between 'Input Inference' and 'Training': 15497


In [ ]:
df = read_csv_file(TRAINING_DATA_FILE)
training_group = set(df.group)
print(f"training data groups: {set(df.group)}")

df = read_csv_file(INFERENCE_INPUT_DATA_FILE)
inference_group = set(df.group)
print(f"inference data groups: {set(df.group)}")

training_group in inference_group

training data groups: {'E02-15', 'F0514e', 'E05-01', 'E02-07', 'E11-05', 'E08-02', 'H0806e', 'E08-03', 'E19-11', 'E29-09', 'E02-11', 'Q1801e', 'E27-30', 'E26-01', 'E02-09', 'E0412e', 'E26-03', 'E17-01', 'E25-01', 'E06-13', 'Z2607e', 'E10-02', 'E17-03', 'E07-08', 'E11-18', 'C0212e', 'D0302e', 'E26-24', 'E10-04', 'E11-15', 'E26-29', 'R0602e', 'E02-05', 'E01-06', 'E11-17', 'E28-19', 'T1904e', 'E06-15', 'E10-10', 'E16-40', 'E28-26', 'U2003e', 'Z2611e', 'E26-27', 'E18-14', 'Z2614e', 'E11-09', 'E03-01', 'E09-07', 'E13-04', 'E08-06', 'E24-02', 'E13-06', 'E01-02', 'T1903e', 'E12-08', 'E21-01', 'Z2612e', 'E10-09', 'E03-02', 'E09-02', 'E04-08', 'E18-16', 'E28-02', 'E07-04', 'E28-01', 'X2503e', 'H0807e', 'E04-06', 'D0305e', 'V2204e', 'E04-01', 'E28-28', 'Q1701e', 'E06-09', 'E19-04', 'E14-04', 'E19-12', 'E27-01', 'E24-04', 'E27-02', 'D0304e', 'E02-13', 'W2451e', 'T1918e', 'E11-02', 'C0215e', 'E20-20', 'E29-02', 'E06-11', 'E08-04', 'E27-29', 'E0401c', 'E09-01', 'E07-05', 'E18-03', 'V2103e', 'Z2604e

False

# Fixing Data Files

## Remapping Training Data

In [1]:
"""
Script: remap_training_data_indices.py
Purpose: Remap `original_index` for labeled training data by aligning it with the
         source training_inference_data using ['yearbook_source', 'group', 'row_id'].
Author: Xiao-Fei Zhang
Date: 2025 Mar 24
"""

import pandas as pd
from pathlib import Path
from utils.read_csv_file import read_csv_file
from project_config import TRAINING_DATA_FILE, TRAINING_INFERENCE_DATA_FILE


def remap_original_index(
    training_data_file: Path | str,
    training_inference_data_file: Path | str,
    output_file: Path | str,
):
    """
    Replaces the 'original_index' of training data by aligning it with the
    original indices from training_inference_data_file based on:
    ['yearbook_source', 'group', 'row_id']

    Args:
        training_data_file: path to the labeled training data (with lost index)
        training_inference_data_file: path to the full unlabeled data (with correct index)
        output_file: path to save the remapped training data
    """
    # Load both DataFrames
    train_df = read_csv_file(training_data_file)
    source_df = read_csv_file(training_inference_data_file)

    # Ensure key columns exist
    key_cols = ["yearbook_source", "group", "row_id"]
    for col in key_cols + ["original_index"]:
        if col not in source_df.columns:
            raise ValueError(f"Missing column '{col}' in source file.")

    for col in key_cols:
        if col not in train_df.columns:
            raise ValueError(f"Missing column '{col}' in training file.")

    # Only keep the mapping keys + original_index
    source_map = source_df[key_cols + ["original_index"]].drop_duplicates()

    # Merge to inject the correct original_index into the training data
    remapped_df = train_df.drop(columns=["original_index"], errors="ignore").merge(
        source_map,
        on=key_cols,
        how="left",
        validate="many_to_one",  # each training row must match at most one row in source
    )

    missing = remapped_df["original_index"].isna().sum()
    if missing > 0:
        print(f"⚠️ Warning: {missing} rows in training data could not be matched.")

    # Save to new file
    remapped_df.to_csv(output_file, index=False)
    print(f"✅ Remapped training data saved to: {output_file}")


output_file = r"C:\github\table_lense\pipeline_data\training_inference\training\training_data_remapped.csv"
remap_original_index(
    training_data_file=TRAINING_DATA_FILE,
    training_inference_data_file=TRAINING_INFERENCE_DATA_FILE,
    output_file=output_file,
)

⚠️ Warning: 362 rows in training data could not be matched.
✅ Remapped training data saved to: C:\github\table_lense\pipeline_data\training_inference\training\training_data_remapped.csv


In [2]:
import pandas as pd
from project_config import PREPROCESSED_ALL_DATA_FILE
from utils.read_csv_file import read_csv_file


file_path = PREPROCESSED_ALL_DATA_FILE

print(file_path)


# Load the file
df = read_csv_file(file_path)

# Group by the merge keys and count occurrences
dup_groups = df.groupby(["yearbook_source", "group", "row_id"]).size()

# Filter to keep only those with duplicates (count > 1)
duplicates = dup_groups[dup_groups > 1].reset_index(name="count")

# Show top 10 duplicates
print(f"🔍 Found {len(duplicates)} duplicated key combinations.")
print(duplicates.head(10))

C:\github\table_lense\pipeline_data\preprocessing\preprocessed_data_all.csv
🔍 Found 0 duplicated key combinations.
Empty DataFrame
Columns: [yearbook_source, group, row_id, count]
Index: []


## deduplicate and reindex training and inference data

In [ ]:
"""
Script: deduplicate_and_reindex_training_data.py
Purpose: Clean up duplicate rows introduced during manual labeling by:
    - Sorting consistently
    - Keeping labeled rows when conflicts arise
    - Rebuilding original_index after cleanup
"""

import pandas as pd
from pathlib import Path
from utils.read_csv_file import read_csv_file

from project_config import TRAINING_INFERENCE_DATA_FILE


def clean_and_reindex_training_data(input_path: Path | str, output_path: Path | str):
    # Load file
    df = read_csv_file(input_path)

    # Define sort order (add more columns if needed)
    sort_cols = ["yearbook_source", "group", "row_id"]
    df = df.sort_values(by=sort_cols).reset_index(drop=True)

    # Mark duplicates based on identifying keys
    def resolve_duplicates(group):
        if len(group) == 1:
            return group

        # If one is labeled and the others are "unlabeled", keep the labeled one
        non_unlabeled = group[group["label"].str.lower() != "unlabeled"]
        if len(non_unlabeled) == 1:
            return non_unlabeled

        # Otherwise, just keep the first one
        return group.iloc[[0]]

    cleaned_df = df.groupby(sort_cols, group_keys=False).apply(resolve_duplicates)

    # Rebuild original_index
    cleaned_df = cleaned_df.reset_index(drop=True)
    cleaned_df["original_index"] = range(len(cleaned_df))

    # Save cleaned file
    cleaned_df.to_csv(output_path, index=False)
    print(f"✅ Cleaned & reindexed file saved to: {output_path}")
    print(f"✅ Final shape: {cleaned_df.shape}")


output_file = r"C:\github\table_lense\pipeline_data\training_inference\training_and_inference_data_deduped.csv"
clean_and_reindex_training_data(
    input_path=TRAINING_INFERENCE_DATA_FILE,
    output_path=output_file,
)

C:\Users\xzhan\AppData\Local\Temp\ipykernel_21804\3367607895.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cleaned_df = df.groupby(sort_cols, group_keys=False).apply(resolve_duplicates)


✅ Cleaned & reindexed file saved to: C:\github\table_lense\pipeline_data\training_inference\training_and_inference_data_deduped.csv
✅ Final shape: (67882, 9)


In [5]:
from pathlib import Path

folder = Path(r"C:\github\table_lense\pipeline_data\output\reconstructed_tables")
file_count = len([f for f in folder.iterdir() if f.is_file()])
print("Number of files:", (file_count - 1) / 3)

Number of files: 1476.0
